**Run the following two cells before you begin.**

In [14]:
%autosave 10

Autosaving every 10 seconds


In [15]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [16]:
# Import the data set
data=pd.read_csv(r'C:\Users\Shridhar\Desktop\cleaned_data.csv')
data                
    

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26659,ecff42d0-bdc6,220000,1,3,1,39,0,0,0,0,...,5003,3047,5000,1000,0,high school,0,1,0,0
26660,99d1fa0e-222b,150000,1,3,2,43,-1,-1,-1,-1,...,8998,129,0,0,0,high school,0,1,0,0
26661,95cdd3e7-4f24,30000,1,2,2,37,4,3,2,-1,...,22000,4200,2000,3100,1,university,0,0,0,1
26662,00d03f02-04cd,80000,1,3,1,41,1,-1,0,0,...,1178,1926,52964,1804,1,high school,0,1,0,0


In [17]:
# Define the sigmoid function

def sigmoid(X):
   Y=1/(1+np.exp(-X))
   return Y


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [18]:
# Create a train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['PAY_1'].values.reshape(-1,1),data['default payment next month'].values,test_size=0.2, random_state=24)


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [19]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [20]:
# Fit the logistic regression model on training data
lr_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Make predictions using `.predict()`
y_pred=lr_model.predict(X_test)

In [22]:
# Find class probabilities using `.predict_proba()`
y_pred_proba = lr_model.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [23]:
# Add column of 1s to features
column_1=np.hstack([np.ones((X_test.shape[0],1)),X_test])
column_1

array([[ 1.,  2.],
       [ 1.,  1.],
       [ 1., -1.],
       ...,
       [ 1., -1.],
       [ 1.,  1.],
       [ 1.,  2.]])

In [25]:
# Get coefficients and intercepts from trained model
print('Coefficients Of Trained Logistic Model',lr_model.coef_)
print('Intercepts of Trained Logistic Model',lr_model.intercept_)
model_coefficient=lr_model.coef_
model_intercept=lr_model.intercept_


Coefficients Of Trained Logistic Model [[0.75064911]]
Intercepts of Trained Logistic Model [-1.39753608]


In [26]:
coefficient_intercept_features=np.concatenate([model_intercept.reshape(1,1),model_coefficient],axis=1)
coefficient_intercept_features

array([[-1.39753608,  0.75064911]])

In [27]:
# Manually calculate predicted probabilities
manual_probability=np.dot(coefficient_intercept_features,np.transpose(column_1))
manual_probability

array([[ 0.10376214, -0.64688697, -2.14818519, ..., -2.14818519,
        -0.64688697,  0.10376214]])

In [29]:
sigmoid_probability=sigmoid(manual_probability)
sigmoid_probability

array([[0.52591729, 0.34369139, 0.10450093, ..., 0.10450093, 0.34369139,
        0.52591729]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [30]:
# Manually calculate predicted classes
pred_class=sigmoid_probability>=0.5

In [33]:
# Compare to scikit-learn's predicted classes
print(pred_class.shape)
print(y_pred.shape)
np.array_equal(pred_class,y_pred.reshape(1,-1))

(1, 5333)
(5333,)


True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [35]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn import metrics
metrics.roc_auc_score(y_test,y_pred_proba[:,1])

0.675865230987299

In [36]:
# Use manually calculated predicted probabilities to calculate ROC AUC
metrics.roc_auc_score(y_test,manual_probability.reshape(manual_probability.shape[1],))

0.675865230987299